# Municipal Population Change in Spain (1996–2024):
## *Analysis of Annual and Multi-Year Intervals*🔁

In [3]:
"""
Notebook: 02_population_variation_analysis.ipynb
Author: Juan Zotes
Created during exploratory phase (Dec 2025)
Last updated: 2026-02-02


Purpose:
    Analyze population variation patterns at the municipal level in Spain
    using cleaned historical census data (Padrón Municipal, 1996–2024).

Scope:
    - Input: Cleaned and harmonized municipal demographic dataset
    - Output: Derived indicators of population change and exploratory results
    - Spatial unit: Municipality (INE codes)
    - Temporal unit: Annual

Analysis focus:
    - Absolute and relative population change
    - Temporal trends and breakpoints
    - Initial exploratory patterns of growth, decline, and stability
    - Preparation of variables for subsequent spatial analysis

Notes:
    - This notebook assumes fully cleaned and standardized input data
    - Results are exploratory and descriptive, not causal
    - No spatial modeling or inference is performed at this stage

Status:
    - Core analytical workflow implemented
    - Indicators considered stable
    - Notebook may be extended with additional metrics or stratifications
"""


'\nNotebook: 02_population_variation_analysis.ipynb\nAuthor: Juan Zotes\nCreated during exploratory phase (Dec 2025)\nLast updated: 2026-02-02\n\n\nPurpose:\n    Analyze population variation patterns at the municipal level in Spain\n    using cleaned historical census data (Padrón Municipal, 1996–2024).\n\nScope:\n    - Input: Cleaned and harmonized municipal demographic dataset\n    - Output: Derived indicators of population change and exploratory results\n    - Spatial unit: Municipality (INE codes)\n    - Temporal unit: Annual\n\nAnalysis focus:\n    - Absolute and relative population change\n    - Temporal trends and breakpoints\n    - Initial exploratory patterns of growth, decline, and stability\n    - Preparation of variables for subsequent spatial analysis\n\nNotes:\n    - This notebook assumes fully cleaned and standardized input data\n    - Results are exploratory and descriptive, not causal\n    - No spatial modeling or inference is performed at this stage\n\nStatus:\n    - 

### 🗂️Import and read the files
After preprocessing in *01_data_cleaning_padron_historico.ipynb*, we can import *01_padron_clean_1996_2024.csv*.

In [4]:
# Import pandas
from pathlib import Path
import pandas as pd

# Base data directory (portable across Windows, Linux, Codespaces)
DATA_DIR = Path(
    r"/workspaces/rural-migration-land-use-spain/data/demography/processed"
)

# File Path
fp_padron_clean = DATA_DIR / "01_padron_clean_1996_2024.csv"

# Read / import the file
padron_clean = pd.read_csv(
    fp_padron_clean,
    sep = ",",
    encoding = "latin1"   # recommended if I plan using the file in QGIS
)

# Check results
print(padron_clean.head())
print(" ")
print(padron_clean.tail())

   Mun_Code     Mun    Cat  Year   Pop
0     44001  Ababuj  Total  2024  74.0
1     44001  Ababuj  Total  2023  70.0
2     44001  Ababuj  Total  2022  72.0
3     44001  Ababuj  Total  2021  76.0
4     44001  Ababuj  Total  2020  77.0
 
        Mun_Code      Mun      Cat  Year     Pop
683587      4103  Zurgena  Mujeres  2001  1062.0
683588      4103  Zurgena  Mujeres  2000  1064.0
683589      4103  Zurgena  Mujeres  1999  1068.0
683590      4103  Zurgena  Mujeres  1998  1069.0
683591      4103  Zurgena  Mujeres  1996  1075.0


#### Check Data Types

In [5]:
padron_clean.dtypes

Mun_Code      int64
Mun             str
Cat             str
Year          int64
Pop         float64
dtype: object

### 🧭 Analysis of Annual and Multi-Year Intervals

This notebook computes the percentage population change for each municipality and category across multiple intervals (1 year, 2 years, … up to the full span). Results are returned in a single tidy table that includes: municipality code, category, year t, interval (k), population at t, population at t-k and the percent variation.


In [6]:
# libs required for the function
import pandas as pd
import numpy as np

def compute_multi_interval_variation(padron_clean,
                                     year_col="Year",
                                     pop_col="Pop",
                                     index_cols=("Mun_Code", "Mun"),
                                     start_year=1996,
                                     end_year=2024):
    """
    Compute percent population variation for multiple interval lengths (k).

    Parameters
    ----------
    padron_clean : pd.DataFrame
        Clean long-format dataframe containing at least index_cols, year_col and pop_col.
    year_col : str
        Column name storing year (int).
    pop_col : str
        Column name storing population (numeric).
    index_cols : tuple
        Columns that uniquely identify municipalities (will be used as the pivot index).
    start_year, end_year : int
        Range of years to consider (inclusive).

    Returns
    -------
    df_all : pd.DataFrame
        Wide dataframe (index = index_cols) containing original year columns and appended
        variation columns named like "pop_1996_1997 (k=1)".
    dict_per_k : dict
        {k: DataFrame} mapping each interval length k to a DataFrame with only that k's variation cols.
    """
    # 0) Basic checks
    if not isinstance(padron_clean, pd.DataFrame):
        raise TypeError("padron_clean must be a pandas DataFrame")

    # ensure year column is integer indexable
    if not pd.api.types.is_integer_dtype(padron_clean[year_col]):
        raise ValueError(f"{year_col} must be integer dtype")

    # ensure population column is numeric
    padron_clean = padron_clean.copy()
    padron_clean[pop_col] = pd.to_numeric(padron_clean[pop_col], errors="coerce")

    # 1) Pivot to wide: rows = municipality id, cols = years
    pivot = (padron_clean
             .pivot_table(index=list(index_cols), columns=year_col, values=pop_col, aggfunc="sum")
             .sort_index(axis=1))

    # 2) Prepare outputs
    df_all = pivot.copy()   # start with year columns (you may drop them later)
    dict_per_k = {}
    k_max = end_year - start_year

    # 3) Loop over k values
    for k in range(1, k_max + 1):
        df_k = pd.DataFrame(index=pivot.index)
        for y in range(start_year, end_year - k + 1):
            y_end = y + k
            # skip pairs if missing
            if (y in pivot.columns) and (y_end in pivot.columns):
                col_start = pivot[y]
                col_end = pivot[y_end]

                # avoid division by zero/NaN: if start is 0 or NaN -> result NaN
                mask_valid = col_start.replace({0: np.nan}).notna()
                variation = (col_end - col_start) / col_start * 100
                variation = variation.where(mask_valid, other=np.nan)

                col_name = f"pop_{y}_{y_end} (k={k})"
                df_k[col_name] = variation
            else:
                continue

        dict_per_k[k] = df_k
        if not df_k.empty:
            df_all = pd.concat([df_all, df_k], axis=1)

    return df_all, dict_per_k


In [7]:
# define k values and call the function (run after function is defined)
k_values = list(range(1, 29))   # k = 1 .. 28 (1996->2024 => 28-year max)
df_all, dict_per_k = compute_multi_interval_variation(padron_clean,
                                                      year_col="Year",
                                                      pop_col="Pop",
                                                      index_cols=("Mun_Code", "Mun"),
                                                      start_year=1996,
                                                      end_year=2024)

### Guardar Todo un csv

In [8]:
DERIVED_DIR = Path(
    r"/workspaces/rural-migration-land-use-spain/data/demography/derived"
)

df_all.to_csv(DERIVED_DIR / "padron_variations_all_k.csv", encoding="latin1", index=True)

### Guardar un csv por k

In [9]:
for k, df_k in dict_per_k.items():
    if not df_k.empty:
        df_k.to_csv(DERIVED_DIR / f"padron_variations_k_{k:02d}.csv", encoding="latin1", index=True)